In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import scipy
#import slycot
import matplotlib.pyplot as plt
from neurosim.models.ssr import StateSpaceRealization as SSR

In [6]:
import pickle
import sys

In [7]:
sys.path.append('../..')

In [8]:
from riccati import check_gdare, solve_gdare

### Positive Realness debugging

In [25]:
with open('test_case3.dat', 'rb') as f:
    varlist = pickle.load(f)
A, C, Cbar, L0 = varlist

In [26]:
check_gdare(A, C, Cbar, L0, tol=1e-2)

True

In [13]:
# Compare solutions of scipy's solve_discrete_are with slycot solver
# Also try implementations of square root filters

In [27]:
Pinf1 = scipy.linalg.solve_discrete_are(A.T, C.T, np.zeros(A.shape), L0, s=Cbar.T)

In [28]:
np.linalg.eigvals(Pinf1)

array([ 1.99953651, -0.78987346, -0.54351242, -0.4711019 ,  0.0407459 ,
       -0.36933667, -0.01314757, -0.02687123, -0.03116439, -0.09048765,
       -0.32908127, -0.12197568, -0.28704822, -0.26301037, -0.22132354,
       -0.18238889, -0.19533339])

In [42]:
Pinf2, rcond, w, S, T = slycot.synthesis.sb02od(A.shape[0], C.shape[0], -A.T, C.T, Q=np.zeros(A.shape), R=-L0, dico='D', L=Cbar.T)

In [43]:
np.linalg.eigvals(Pinf2)

array([-1.99953651,  0.78987346,  0.54351242,  0.4711019 , -0.0407459 ,
        0.36933667,  0.01314757,  0.02687123,  0.03116439,  0.09048765,
        0.32908127,  0.12197568,  0.28704822,  0.26301037,  0.22132354,
        0.18238889,  0.19533339])

In [27]:
# Directly implement...

In [31]:
AA, BB, alpha, beta, Q, Z = solve_gdare(A, C, Cbar, L0)

In [32]:
geig = np.abs(np.divide(alpha, beta))

In [33]:
iuc = np.where(np.abs(geig) < 1)[0]

In [35]:
U1 = Z[0:A.shape[0], iuc]
U2 = Z[A.shape[0]:, iuc]

In [36]:
Pinf3 = scipy.linalg.solve(U1.T, U2.T)

In [37]:
np.linalg.eigvals(Pinf3)

array([ 1.10309849e+02,  1.32304383e+01, -3.51228231e+00,  5.05315906e+00,
        3.99121931e+00,  1.87774018e-02,  5.39526330e-01,  6.81320123e-01,
        2.36257337e+00,  2.31036455e+00,  2.05630755e+00,  1.18660739e+00,
        1.26179059e+00,  1.40930774e+00,  1.76403759e+00,  1.57378104e+00,
        1.62737429e+00])

In [59]:
# Does it solve the Riccati equation?
from riccati import discrete_generalized_riccati, discrete_riccati

In [44]:
np.linalg.norm(Pinf2 - discrete_generalized_riccati(Pinf2, A, C, Cbar, L0))

2.636863275618374e-14

In [46]:
np.linalg.eigvals(Pinf2)

array([-1.99953651,  0.78987346,  0.54351242,  0.4711019 , -0.0407459 ,
        0.36933667,  0.01314757,  0.02687123,  0.03116439,  0.09048765,
        0.32908127,  0.12197568,  0.28704822,  0.26301037,  0.22132354,
        0.18238889,  0.19533339])

In [47]:
np.linalg.eigvals(L0 - C @ Pinf2 @ C.T)

array([ 4.74189932, -0.26476504,  1.80749544,  1.5401542 ,  0.62185488,
        0.71584091,  0.9188448 ,  1.12751044,  1.01783403,  1.07531563])

In [50]:
np.linalg.pinv(L0 - C @ Pinf2 @ C.T)

array([[-0.41109591,  0.7170694 , -1.68620944,  0.34748773,  0.17251269,
         0.07022032, -0.19105052,  1.377519  ,  0.23344722,  0.39175148],
       [ 0.7170694 ,  0.64037057,  0.43723568, -0.1844536 , -0.04152925,
         0.16914235,  0.30652485, -0.82333773, -0.21327981, -0.24290065],
       [-1.68620944,  0.43723568, -0.17101315,  0.4087879 ,  0.09318001,
        -0.3249468 ,  0.02775428,  1.12407058,  0.09478938,  0.45936201],
       [ 0.34748773, -0.1844536 ,  0.4087879 ,  0.79898573, -0.01772761,
         0.23043358,  0.01909752, -0.49220315, -0.0620754 , -0.09592097],
       [ 0.17251269, -0.04152925,  0.09318001, -0.01772761,  0.91950066,
        -0.10641036, -0.01099426, -0.02747409,  0.05647644, -0.02867474],
       [ 0.07022032,  0.16914235, -0.3249468 ,  0.23043358, -0.10641036,
         0.88667452,  0.02283101, -0.08276278, -0.09273402, -0.02531967],
       [-0.19105052,  0.30652485,  0.02775428,  0.01909752, -0.01099426,
         0.02283101,  0.50859417,  0.35598724

In [30]:
from subspaces import estimate_autocorrelation, SubspaceIdentification, IteratedStableEstimator

In [86]:
iter = 0
# Checking for failure of PSD of the riccati soliution
while True:
    state_dim = 20
    obs_dim = 10
    A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))
    while max(np.abs(np.linalg.eigvals(A))) > 0.99:
        A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))

    C = scipy.stats.ortho_group.rvs(state_dim)[:, 0:obs_dim].T
    ssr = SSR(A=A, B=np.eye(A.shape[0]), C=C)
    ccm0 = ssr.autocorrelation(5)
    y = ssr.trajectory(int(1e3))
    ccm1 = estimate_autocorrelation(y, 5)   
    ssid1 = SubspaceIdentification(T=3, estimator=IteratedStableEstimator, score='BIC', obs_regressor='OLS')
    _, _, _, _ = ssid1.identify(y)
    # Pinf = scipy.linalg.solve_discrete_are(A.T, C.T, Q, R, s=S)
    # eig = np.linalg.eigvals(Pinf)
    # eig = np.linalg.eigvals(PInf)
    # if np.any(eig < 0):
    #     print('Non PSD P found!')
    #     break
    # L0 = 0.5 * (L0 + L0.T)
    # Pinf2 = scipy.linalg.solve_discrete_are(A.T, -C.T, np.zeros(A.shape), -L0, s=Cbar.T)
    # eig = np.linalg.eigvals(Pinf)
    # if np.any(eig < 0):
    #     print('Non PSD P- found!')
    #     break

    iter += 1
    print(iter)


True
True
True
True
True
pr correction employed
True
pr correction employed
True
pr correction employed
True
pr correction employed
True
> /home/akumar/nse/neural_control/subspaces.py(197)filter_log_likelihood()
    195             pdb.set_trace()
    196 
--> 197         if np.any(np.isinf(P)) or np.any(np.isnan(P)):
    198             pdb.set_trace()
    199 



BdbQuit: 

In [ ]:
# So we always get a positive definite solution to the Riccati equation. They key is then finding a stable iteration th at converges to this solution

In [ ]:
# Can we code up a test of the necessary/sufficient conditions for convergence provided by Kailath?

In [66]:
import pickle

In [76]:
with open('test_case5.dat', 'rb') as f:
    varlist = pickle.load(f)
    A, C, Q, R, S, Pinf = varlist

In [82]:
np.linalg.eigvals(Pinf)

array([2.27246925, 1.91094207, 1.60917569, 1.50984817, 1.2329283 ,
       1.07455583, 0.89707979, 0.15340883, 0.24352311, 0.26783781,
       0.31939546, 0.38168371, 0.71585725, 0.71223314, 0.44590874,
       0.47804822, 0.51379959, 0.5465913 , 0.64529787, 0.62706115])

In [77]:
np.linalg.eigvals(R)

array([9.32021859e-01, 5.51274788e-01, 3.09405093e-01, 1.68205128e-01,
       9.09957396e-02, 3.01910013e-02, 2.42523470e-02, 1.90557441e-02,
       1.81358557e-04, 1.03698400e-03])

In [79]:
np.linalg.eigvals(Q)

array([1.01447406, 0.91623909, 0.82377518, 0.79935936, 0.73927198,
       0.715388  , 0.65455949, 0.04751026, 0.57177943, 0.51679994,
       0.46731078, 0.44385444, 0.38317171, 0.33108076, 0.30190755,
       0.12495745, 0.14824874, 0.23030189, 0.19921325, 0.1769245 ])

In [81]:
np.linalg.eigvals(np.block([[Q, S], [S.T, R]]))

array([1.06800411e+00, 9.72457336e-01, 8.91357265e-01, 8.18361982e-01,
       8.04883566e-01, 7.58610946e-01, 7.22720929e-01, 6.99842394e-01,
       5.69737266e-01, 5.38708309e-01, 4.96664859e-01, 4.64851776e-01,
       4.35651556e-01, 4.02190707e-01, 3.60295511e-01, 3.15863940e-01,
       2.89882666e-01, 2.30400092e-01, 2.11993224e-01, 1.76432054e-01,
       1.33698041e-01, 1.12728400e-01, 8.63145118e-02, 6.63591892e-02,
       4.55749125e-02, 7.84944446e-04, 1.30081639e-04, 1.36688720e-02,
       2.52717781e-02, 1.93066562e-02])

In [83]:
# Check unit circle controllability
Fs = A - S @ np.linalg.inv(R) @ C

In [85]:
np.abs(np.linalg.eigvals(Fs))

array([2.83721894, 2.83721894, 1.93052056, 1.82502613, 1.09348154,
       1.09348154, 1.02699262, 1.02699262, 0.98720627, 0.98720627,
       1.07006834, 1.07006834, 1.15059438, 0.55515037, 0.55515037,
       0.51907198, 0.51907198, 0.76467797, 0.76467797, 0.56455857])

In [84]:
# Try solving the error Riccati equation
norm_diff = []

Q = Pl

for i in range(1000):
    Q_ = discrete_riccati(Q, A, B, C, L0, S=D.T)
    norm_diff.append(np.linalg.norm(Q - Q_))
    Q = Q_

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 13)

### Use the pykalman filter/smoother

In [6]:
from pykalman.sqrt.cholesky import CholeskyKalmanFilter
from pykalman import KalmanFilter

In [ ]:
# Wrapper around pykalman's Kalman Filter to enable correlations between the noises
# This requires R > 0 (see Kailath 9.5.1)
class KalmanFilterWrapper(KalmanFilter):
    def __init__(self, A, C, Q, R, S, initial_state_mean=None, initial_state_covariance=None, random_state=None):

        # Modify A and Q to include S
        A = A - S @ np.linalg.inv(R) @ C
        Q = Q - S @ np.linalg.inv(R) @ S.T


    def filter():



### Test ML inference

In [9]:
from em import StableStateSpaceML

In [10]:
state_dim = 20
obs_dim = 10
A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))
while max(np.abs(np.linalg.eigvals(A))) > 0.99:
    A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))

C = scipy.stats.ortho_group.rvs(state_dim)[:, 0:obs_dim].T
ssr = SSR(A=A, B=np.eye(A.shape[0]), C=C)

In [15]:
sssml = StableStateSpaceML()

In [12]:
y = ssr.trajectory(int(1e4))

In [17]:
sssml.fit(y, state_dim=20)

> /home/akumar/nse/neural_control/em.py(83)fit()
     81             ssid = SubspaceIdentification(estimator=IteratedStableEstimator, obs_regressor='OLS', **init_kwargs)
     82             pdb.set_trace()
---> 83             A, C, Cbar, L0, Q, R, S = ssid.identify(y, order=state_dim)
     84             Ainit = A
     85             Cinit = C

20
*** TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'
(10001, 10)
*** NameError: name 'self' is not defined


BdbQuit: 